In [1]:
!pip install eli5==0.13.0
!pip lightgbm
# 한글 폰트 사용을 위한 라이브러리입니다.
!apt-get install -y fonts-nanum

ERROR: unknown command "lightgbm"
Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20180306-3).
0 upgraded, 0 newly installed, 0 to remove and 26 not upgraded.


In [3]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

In [4]:
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import train_test_split
import lightgbm

# 필요한 데이터를 load
total_path = "../total-data_ver2.csv"

dt_total = pd.read_csv(total_path)

# is_test 컬럼 값이 0인 행을 train 데이터셋으로 저장
df_train = dt_total[dt_total['is_test'] == 0]

# is_test 컬럼 값이 1인 행을 test 데이터셋으로 저장
df_test = dt_total[dt_total['is_test'] == 1]

In [5]:
# '계약년월'에서 '연도' 추출 후 int 형으로 변환하여 새로운 열 추가
df_train = df_train.assign(year=df_train['계약년월'].astype('str').str[:4].astype(int)).query('year > 2020')

# '계약년월'에서 '월' 추출 후 5로 나눈 값을 새로운 열로 추가
df_train, df_test = map(lambda df: df.assign(계약월=lambda x: (x['계약년월'] % 100) // 5), [df_train, df_test])

# 타겟 변수 생성
df_train['price'] = df_train.groupby(['아파트명', '계약월'])['target'].transform('mean')

# 중복 데이터 제거
df_train = df_train.drop_duplicates(subset=['아파트명', '계약월'])

In [6]:
# 특징 변수와 타겟 변수 설정
X = df_train[['전용면적', 'x']]
y = df_train['price']
test = df_test[['전용면적', 'x']]

# 데이터를 학습 데이터와 검증 데이터로 분리합니다.
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=2023)

# 모델 초기화
model = lightgbm.LGBMRegressor(n_estimators=100, learning_rate=0.01, max_depth=5)

# 모델 학습
model.fit(X_train, y_train)

# 검증 데이터에 대한 예측
val_pred = model.predict(X_val)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000391 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 510
[LightGBM] [Info] Number of data points in the train set: 8350, number of used features: 2
[LightGBM] [Info] Start training from score 97758.881027
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N

In [7]:
# RMSE 계산
rmse = sqrt(mean_squared_error(y_val, val_pred))
print("RMSE: ", rmse)

RMSE:  67394.02934576674


In [8]:
# MSE 계산
mse = mean_squared_error(y_val, val_pred)
print("MSE: ", mse)

MSE:  4541955191.45807


In [9]:
# 테스트 데이터에 대한 예측
pred = model.predict(test)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [10]:
# 예측 결과 저장
pd.DataFrame(pred.astype(int), columns=['target']).to_csv('simple.csv', index=False)

# 저장된 파일 확인
print(pd.read_csv('simple.csv'))

      target
0      84953
1      64968
2      85798
3      68725
4      68725
...      ...
9267   99532
9268   99532
9269   99532
9270  115138
9271  115138

[9272 rows x 1 columns]
